In [1]:
import igraph as ig
import numpy as np
from sympy.solvers import nsolve
from sympy import *

In [2]:
import scipy as sp
from scipy import stats

In [3]:
from scipy.stats import rankdata

In [4]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [5]:
from shapely.geometry import Point,MultiPoint,LineString,Polygon

In [6]:
import powerlaw as pl
%matplotlib inline
import matplotlib.pyplot as plt

In [7]:
from scipy.spatial import distance
def dist(v1,v2):
    dist = distance.euclidean((v1[0],v1[1]),(v2[0],v2[1]))
    return dist

In [8]:
from scipy import special
from scipy import stats
import pandas as pd

In [11]:
aggragate = pd.read_csv('final_table_ca.csv')

In [13]:
x_coord = np.array((aggragate['Lon'].tolist(),aggragate['Lat'].tolist())).T
y_coord = np.array((aggragate['Lon'].tolist(),aggragate['Lat'].tolist())).T

In [14]:
population = ((aggragate['pop'].values).reshape(111,1)).tolist()

In [15]:
coords = np.concatenate([x_coord,y_coord],axis=1)

In [16]:
n = 111
alldist = []
A = np.zeros((n,n))
A_binom = np.zeros((n,n))
count=0
for i in range(0,n):
    for j in range(i+1,n):
        m = (5*10**-7)*(population[i][0]*population[j][0])/haversine(x_coord[i][0], x_coord[i][1], y_coord[j][0], y_coord[j][1])**2#(dist(coords[i],coords[j]))**2
        A[i][j] = m
        A[j][i] = m
        h = (np.random.binomial(111, 0.01503695919007753, 1))/110
        A_binom[i][j] = h
        A_binom[j][i] = h

In [17]:
Dt = np.sum(A,axis=1).tolist()
DD = np.diag(Dt)
L = A - DD

Dt_binom = np.sum(A_binom,axis=1).tolist()
DD_binom = np.diag(Dt_binom)
L_binom = A_binom - DD_binom

In [18]:
def bins_counts(array,a,b,c):
    count, bins = np.histogram(array, bins=np.logspace(a,b,c),density=1)
    t =[]
    for i in range(len(count)):
        t.append((bins[i + 1] + bins[i])/float(2))
    return t,count

In [20]:
d_frac = aggragate['d_frac'].tolist()

In [21]:
#np.random.shuffle(d_frac)
x0_cf = []
y0_cf = []
c=[]#### status parameter
a_12 = []
a_21 = []

for i in range(111):
    if d_frac[i] > 0.5:
        x0_cf.append(1.2)
        y0_cf.append(0.4)
        c.append(0.5)
        a_12.append(21)
        a_21.append(0.0)
    if d_frac[i] <= 0.5:
        x0_cf.append(0.4)
        y0_cf.append(1.2)
        c.append(-0.5)
        a_12.append(0.0)
        a_21.append(21)
aggragate['c'] = c
aggragate['x0'] = x0_cf
aggragate['y0'] = y0_cf
aggragate['a12'] = a_12
aggragate['a21'] = a_21

In [22]:
cc = (aggragate['c'].values).reshape(111,1)
a_12 = (aggragate['a12'].values).reshape(111,1)
a_21 = (aggragate['a21'].values).reshape(111,1)

In [23]:
deltat=10.**(-3.)
def fun(e,f):
    l1 = np.matmul(L,e)
    l5 = np.matmul(L,f)
    l3 = np.matmul(L,(e*f))
    return (e + deltat*(e*(1-e) + cc*e*f +l1*d1 + l3*a_12), f + deltat*(f*(1-f) - cc*e*f + l5*d2 + l3*a_21))

In [24]:
d1 = 0.01
d2 = 0.01

In [25]:
t= 140000#Pasos temporales totales
tiempo=np.linspace(0,t,t+1) 

In [ ]:
evolution_2x = []
evolution_2y = []
for i in range(5):
    up = []
    vp = []
    uavg=[]
    x_0   = 1.2#aggragate['x0'].tolist()
    y_0  = 0.4#aggragate['y0'].tolist()
    for n in range((111)):
        up.append(x_0 + 0.1*x_0*(2.*np.random.rand()-1))
        vp.append(y_0 + 0.1*y_0*(2.*np.random.rand()-1))
    upp = np.array([up]).T
    vpp = np.array([vp]).T
    ui = upp
    vi = vpp
    #uavg= []
    for j in tiempo:
        u =(fun(ui,vi))[0]
        v=(fun(ui,vi))[1]
        ui = u
        vi = v
        uavg.append(np.average(ui))
    evolution_2x.append(ui)
    evolution_2y.append(vi)
    print(i)

In [ ]:
aggragate['xi_sim'] = np.mean(evolution_2x,axis=0)
aggragate['yi_dim'] = np.mean(evolution_2y,axis=0)

In [ ]:
new=pd.DataFrame()
new['xi_sim'] = (aggragate.groupby('districts').apply(lambda x: np.average(x['xi_sim']/(x['xi_sim']+x['yi_sim']), weights=x['pop']))).tolist()


new['yi_sim'] = (aggragate.groupby('districts').apply(lambda x: np.average(x['yi_sim']/(x['xi_sim']+x['yi_sim']), weights=x['pop']))).tolist()




In [ ]:
sp.stats.pearsonr(new['yi_fix_emp'].tolist(),new['s_frac'].tolist())

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(2*6.4, 4.8))
font0 = FontProperties()
font1 = font0.copy()
font1.set_weight('bold')
font1.set_size('large')
font1.set_family('sans-serif')

axes[0].text(0.5, 1.45, 
                'German',
        verticalalignment='center', horizontalalignment='center',
        
        color='black',fontproperties=font1,fontsize=20,transform=axes[0].transAxes, **hfont)
ka_agg_1.plot(ax=axes[0], column='xi_e_s_1', cmap='Oranges', edgecolor='black',vmin=0,vmax=1)

axes[0].set_yticklabels([])
axes[0].set_xticklabels([])
axes[0].set_yticks([])
axes[0].set_xticks([])

axes[0].spines['right'].set_visible(False)
axes[0].spines['top'].set_visible(False)
axes[0].spines['bottom'].set_visible(False)
axes[0].spines['left'].set_visible(False)

axes[0].text(0.1,0.035,r'$\rho^G_p = 0.89$',fontsize=20,transform=axes[0].transAxes, **hfont)



axes[0].text(0.05, 0.95, '(a)',
        verticalalignment='center', horizontalalignment='center',
        
       color='black',fontproperties=font1,fontsize=20,transform=axes[0].transAxes, **hfont)

#########
axes[1].text(0.5, 1.45, 
                'Slovenian',
        verticalalignment='center', horizontalalignment='center',
        
        color='black',fontproperties=font1,fontsize=20,transform=axes[1].transAxes, **hfont)
ka_agg_1.plot(ax=axes[1], column='yi_e_s_1', cmap='Purples', edgecolor='black',vmin=0,vmax=1)

axes[1].set_yticklabels([])
axes[1].set_xticklabels([])
axes[1].set_yticks([])
axes[1].set_xticks([])

axes[1].spines['right'].set_visible(False)
axes[1].spines['top'].set_visible(False)
axes[1].spines['bottom'].set_visible(False)
axes[1].spines['left'].set_visible(False)

axes[1].text(0.1,0.035,r'$\rho^S_p = 0.89$',fontsize=20,transform=axes[1].transAxes, **hfont)



axes[1].text(0.05, 0.95, '(b)',
        verticalalignment='center', horizontalalignment='center',
        
       color='black',fontproperties=font1,fontsize=20,transform=axes[1].transAxes, **hfont)





vmin = 0
vmax = 1
fig.tight_layout()
sm = plt.cm.ScalarMappable(cmap='Oranges', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
sm_1 = plt.cm.ScalarMappable(cmap='Purples', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm_1._A = []
cbaxes = fig.add_axes([0.4735, 0.04, 0.012, 0.9]) 
cbar = fig.colorbar(sm,cax=cbaxes,orientation='vertical',ticks=[0, 0.5, 1])
cbar.ax.tick_params(labelsize=20)
cbaxes = fig.add_axes([.97, 0.04, 0.012, 0.9]) 
cbar = fig.colorbar(sm_1,cax=cbaxes,orientation='vertical',ticks=[0, 0.5, 1])
cbar.ax.tick_params(labelsize=20)



plt.savefig('carinthia_extra_map_1.pdf',bbox_inches='tight')